<a href=https://m-sender.github.io/ServiceLearning>Click here for live site!</a>
<h1><b>Max Sender and Sam Traylor</b></h1>
<h2>Data set </h2>
<h3>Data set link: <a href=https://data.nola.gov/Public-Safety-and-Preparedness/Calls-for-Service-2021/3pha-hum9>Calls for service 2021</a></h3> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;What this data set is a collection of 9-1-1 calls in 2021 in the New Orleans Area. This set contains basic things such as the type of incident, where it was, the police department, and timing, and more. 

<h2>Questions</h2>
<h4>We find this data set to be very insightful and can answer a lot of different questions. 
One route we can take is analyzing the data set to focusing on emergency response and answer questions regarding that. If this route is chosen, another dataset that could be of use is <a href=https://data.nola.gov/dataset/Police-Zones/fngt-zkj9> Police Zone Information</a>. This lets us expand our questions to answer more zone and area specific questions. Questions that we can answer going this route are:</h4> 
<ul>
<li>Average response time by incident?</li>
    -We could extract a column of response times (using TimeArrive - TimeDispatch values) and take the mean of those values for each type of incident
<li>Average response time by zone/area?</li>
    -Using that same response time column, we could look at the means across different areas (using the police district or zip column of this dataset)
<li>Average response time by incident in specific areas?</li>
    -Using the results of the last question, we could further specify the avergage response time across incident type column values AND area column values.
<li>Using zone information and response time, determine "holes" in the zones where response time is higher than the norm or where the area has an increase in crime due to the response times.</li>
    -We could use measures of variance like the standard deviation from average response time, which would allow us to identify 'holes' wherever the response time is far higher than average.
</ul>
<h4>Another route we can go with the data is focusing more on the crime aspect of the data set. This route will be more focused on answering questions about crime in specific areas instead of the emergency response.</h4>
<ul>
<li>Average response time by area?</li>
<li>Most frequent crimes in specific areas?</li>
    -By getting the value counts of each type of crime in each area we can generalize patterns like violent crime happening more in one area, theft in another, etc
<li>Most frequest crimes by time of day in conjunction with a specific area?</li>
    -Get the value counts of each different crime for each time of day (we could categorize into several-hour windows like afternoon, evening, night, late night)
<li>Using frequency, determine safer or more dangerous areas depending on the nature of the crimes that are most frequent there.</li>
    -Analyzing the possible values for entries in the TypeText column would allow us to differentiate violent (assault and/or battery, sexual assualt, domestic disturbance) from nonviolent (theft, fireworks, burglar alarms) incidents, 2 categories which crime in each area could now be fit into.
</ul>
<h4>There are more routes we can choose from and more questions will come to mind upon further analysis of the datasets. A combination of multiple routes will most likely render the most promising and insightful results.</h4>
<h2>Collaboration plan:</h2>
<p>We plan to collaborate via meetings over zoom, and store our data in a shared github. Any particular challenges that have to be solved in a pair-programming setting will be dealt with using live share on vscode.</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [6]:
df_Calls = pd.read_csv("../data/Calls_for_Service_2021.csv")
df_zones = pd.read_csv("../data/Police_Zones_data.csv")


In [5]:
print(df_Calls.columns)
df_Calls.head(10)

Index(['NOPD_Item', 'Type', 'TypeText', 'Priority', 'InitialType',
       'InitialTypeText', 'InitialPriority', 'MapX', 'MapY', 'TimeCreate',
       'TimeDispatch', 'TimeArrive', 'TimeClosed', 'Disposition',
       'DispositionText', 'SelfInitiated', 'Beat', 'BLOCK_ADDRESS', 'Zip',
       'PoliceDistrict', 'Location'],
      dtype='object')


,NOPD_Item,Type,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
0,A0000221,22A,AREA CHECK,1K,22A,AREA CHECK,1K,3668804,546585,01/01/2021 12:01:28 AM,...,01/01/2021 12:01:28 AM,01/01/2021 12:40:31 AM,NAT,Necessary Action Taken,Y,3I04,Vicksburg St & Brooks St,70124,3,POINT (-90.10764787 29.99729994)
1,A0000321,62A,"BURGLAR ALARM, SILENT",1A,62A,"BURGLAR ALARM, SILENT",2E,3673216,521932,01/01/2021 12:01:34 AM,...,01/01/2021 03:43:58 AM,01/01/2021 03:51:24 AM,NAT,Necessary Action Taken,N,6Q02,036XX Baronne St,70115,6,POINT (-90.09455243 29.92938301)
2,A0000421,22A,AREA CHECK,1K,22A,AREA CHECK,1K,3682049,530554,01/01/2021 12:01:47 AM,...,01/01/2021 12:01:47 AM,01/01/2021 03:03:53 AM,NAT,Necessary Action Taken,Y,8B01,Decatur St & Iberville St,None,8,POINT (-90.06636912 29.95282347)
3,A0000521,94F,FIREWORKS,1A,94F,FIREWORKS,2J,3704326,518525,01/01/2021 12:02:13 AM,...,NaN,01/01/2021 12:17:36 AM,VOI,VOID,N,4D02,055XX Sutton Pl,70131,4,POINT (-89.9964721 29.91905338)
4,A0000621,94,DISCHARGING FIREARM,1A,94,DISCHARGING FIREARM,2D,3702356,554005,01/01/2021 12:02:14 AM,...,NaN,01/01/2021 07:08:48 AM,NAT,Necessary Action Taken,N,7E01,Lonely Oak Dr & Selma St,70126,7,POINT (-90.00138771 30.01667289)
5,A0000721,22A,AREA CHECK,1K,22A,AREA CHECK,1K,3689294,523714,01/01/2021 12:02:37 AM,...,01/01/2021 12:02:37 AM,01/01/2021 01:00:39 AM,NAT,Necessary Action Taken,Y,4A05,Thayer St & Shepard St,70114,4,POINT (-90.04373626 29.93379453)
6,A0000821,94F,FIREWORKS,1A,94F,FIREWORKS,2J,3685751,537403,01/01/2021 12:03:15 AM,...,NaN,01/01/2021 12:03:42 AM,NAT,Necessary Action Taken,N,5B03,013XX Mandeville St,70117,5,POINT (-90.05443947 29.971544)
7,A0000921,94,DISCHARGING FIREARM,2D,94,DISCHARGING FIREARM,2D,3699517,533522,01/01/2021 12:04:11 AM,...,01/01/2021 12:59:57 AM,01/01/2021 01:06:54 AM,GOA,GONE ON ARRIVAL,N,5E03,011XX Blk Tupelo St,70117,5,POINT (-90.01110609 29.96044253)
8,A0001021,94,DISCHARGING FIREARM,1A,94,DISCHARGING FIREARM,2D,3684916,536850,01/01/2021 12:04:19 AM,...,01/01/2021 01:35:08 AM,01/01/2021 01:56:55 AM,RTF,REPORT TO FOLLOW,N,5A05,012XX Blk Elysian Fields Ave,70116,5,POINT (-90.05709394 29.97004982)
9,A0001121,94F,FIREWORKS,1A,103F,FIGHT,2D,3676697,526433,01/01/2021 12:06:10 AM,...,01/01/2021 03:09:39 AM,01/01/2021 03:22:54 AM,GOA,GONE ON ARRIVAL,N,6H02,016XX S Liberty St,70113,6,POINT (-90.08341098 29.94165311)


In [9]:
df_zones

,the_geom,OBJECTID,Zone,District,Shape_Length,Shape_Area
0,MULTIPOLYGON (((-90.066369220964 29.9528235013...,1883,8C,8,6792.167688,2.012343e+06
1,MULTIPOLYGON (((-90.096467011995 29.9414493437...,1855,6K,6,8518.748545,4.324270e+06
2,MULTIPOLYGON (((-90.100235367426 29.9209080675...,1860,6P,6,11703.354343,7.651238e+06
3,MULTIPOLYGON (((-90.108388478789 29.9349666704...,1784,2I,2,14995.498757,1.318799e+07
4,MULTIPOLYGON (((-90.065506929482 29.9391674081...,1847,6C,6,13163.392602,5.747707e+06
...,...,...,...,...,...,...
121,MULTIPOLYGON (((-90.111520545347 29.9527465406...,1795,2U,2,13072.875152,9.234422e+06
122,MULTIPOLYGON (((-90.058618074754 29.9913549365...,1819,3X,3,19241.277584,1.299197e+07
123,MULTIPOLYGON (((-90.040725013307 29.9469852425...,1827,4G,4,14627.126145,1.251777e+07
124,MULTIPOLYGON (((-90.058944234626 29.9502226780...,1882,8B,8,10947.349649,6.273422e+06
